# eidein

Interactive tool for dimensionality reduction of astronomical data and their visualisation.

## TODO List

1. implement BNN and prediction uncertanties
1. subset (lasso) selection (widgets.SelectMultiple)
1. divide features (for dimensionality reduction) and spectra (for visualisation, ploting function)
1. scale to Big Data

In [ ]:
%matplotlib widget

from astropy.modeling import models, fitting
import h5py
import matplotlib.pyplot as plt
import numpy as np

from eidein import Eidein
import plot

## Data

In [ ]:
SIZE = 2048

with h5py.File("data/dataset.hdf5", "r") as datafile:
    rnd_idx = np.random.randint(datafile["id_tr"].shape[0], size=SIZE)
    ids_rnd = datafile["id_tr"][:][rnd_idx]
    X_rnd = datafile["X_tr"][:][rnd_idx]
    z_rnd = datafile["z_tr"][:][rnd_idx].reshape(-1, 1)
ids_rnd.shape, X_rnd.shape, z_rnd.shape

In [ ]:
fig, ax = plt.subplots()
plot.spectrum(ax, X_rnd[0], label="z = " + str(z_rnd[0, 0]))
plot.redshift(ax, z_rnd[0, 0])
ax.legend()

## Redshift without Continuum Normalisation

In [ ]:
Eidein(ids_rnd, X_rnd, z_rnd)

## Linear Fit to Continuum

In [ ]:
fit = fitting.LinearLSQFitter()
line_init = models.Linear1D()
fitted_line = fit(line_init, plot.WAVE, X_rnd[0])
continuum = fitted_line(plot.WAVE)

fig, ax = plt.subplots()
plot.spectrum(ax, X_rnd[0], label="z = " + str(z_rnd[0, 0]))
plot.redshift(ax, z_rnd[0, 0])
plot.spectrum(ax, continuum)
ax.legend()

fitted_line.slope.value, fitted_line.intercept.value

In [ ]:
fit = fitting.LinearLSQFitter()
line_init = models.Linear1D()

slopes = np.zeros((SIZE, 1))
intercepts = np.zeros((SIZE, 1))
for i, x in enumerate(X_rnd):
    fitted_line = fit(line_init, plot.WAVE, x)
    slopes[i, 0] = fitted_line.slope.value
    intercepts[i, 0] = fitted_line.intercept.value

Eidein(ids_rnd, X_rnd, slopes)

## Redshift with Continuum Normalisation

In [ ]:
fit = fitting.LevMarLSQFitter()
line_init = models.Chebyshev1D(3)

X_norm = np.zeros_like(X_rnd)
for i, x in enumerate(X_rnd):
    fitted_line = fit(line_init, plot.WAVE, x)
    X_norm[i] = X_rnd[i] - fitted_line(plot.WAVE)

fig, ax = plt.subplots()
i = np.random.randint(SIZE)
plot.spectrum(ax, X_rnd[i], label="original")
plot.spectrum(ax, X_norm[i], label="normalised")
ax.legend()

In [ ]:
Eidein(ids_rnd, X_norm, z_rnd)